<a href="https://colab.research.google.com/github/DDoeuiGongju/DoItDeeplearning/blob/main/09_%EC%88%9C%ED%99%98_%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 순차 데이터와 순환 신경망

### 순차 데이터
- 순차 데이터: 샘플에 순서가 있는 데이터
    - 스텝: 모델에서 순차 데이터를 처리하는 각 단계
- 종류
    - 텍스트: 대표적인 순차 데이터
    - 시계열 데이터: 순차 데이터로 일정 시간 간격으로 배치된 데이터 
- 완전 연결 신경망이나 합성곱 신경망은 이전의 샘플에 대한 정보를 유지하지 않고, 현재 샘플 처리 시 이전의 샘플을 고려하지 않는다.
    - 이러한 문제를 해결하기 위해 순환 신경망 개발 

### 순환 신경망
- 순환 신경망: 뉴런의 출력이 순환되는 신경망으로 은닉층의 출력이 다시 은닉층의 입력으로 사용된다.
- 층이나 뉴런을 셀이라 부른다. 셀의 출력을 은닉 상태(hidden state)라 부른다. 현재의 은닉 상태를 계산하기 위해 이전 다음 스텝의 은닉 상태를 사용한다.
- 셀에서 수행되는 계산은 지금까지 공부한 정방향 계산에서 이전 타임 스텝의 은닉 상태와 곱하는 가중치가 하나 더 추가된 것이다.
- 활성화 함수로 Hyperbolic tangent함수가 많이 사용된다.


**<순환층의 정방향 계산>**  
- $ Z = XW_x + H_pW_h + b $  
X=입력, H=은닉상태, W_x W_h=가중치, b=절편
- $ H = tanh(Z) $  

**<출력층의 정발향 계산>**
- $ Z = HW + b $
- $ A = sigmoid(Z) $

**<역방향 계산>**
- https://www.youtube.com/watch?v=iAv6yA7hv2o  
14:50~24:30 내용 참고



### 순환 신경망으로 텍스트 분류
- IMDB dataset: 인터넷 영화 DB에서 수집란 영화 리뷰 데이터
- 리뷰가 긍정인지 부정인지 판별하는 이진 분류 문제 해결

In [8]:
# 1. 텐서플로우에서 IMDB불러오기
import numpy as np
from tensorflow.keras.datasets import imdb

# skip_top: 가장 많이 등장한 단어들 중 건너뛸 단어 개수
# num_words: 훈련에 사용할 단어 개수
(x_train_all, y_train_all), (x_test, y_test) = imdb.load_data(skip_top=20, num_words=100)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [9]:
print(x_train_all.shape, y_train_all.shape)

(25000,) (25000,)


In [10]:
# 샘플의 정수는 영단어를 고유한 정수에 1:1대응한 BoW(Bag of Word)이다.
# 가장 많이 등하는 영단어 20개를 건너뛰고 100개의 단어만 선택했으므로 BoW에 없는 단어를 의미하는 2가 많다.
print(x_train_all[0])

[2, 2, 22, 2, 43, 2, 2, 2, 2, 65, 2, 2, 66, 2, 2, 2, 36, 2, 2, 25, 2, 43, 2, 2, 50, 2, 2, 2, 35, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 39, 2, 2, 2, 2, 2, 2, 38, 2, 2, 2, 2, 50, 2, 2, 2, 2, 2, 2, 22, 2, 2, 2, 2, 2, 22, 71, 87, 2, 2, 43, 2, 38, 76, 2, 2, 2, 2, 22, 2, 2, 2, 2, 2, 2, 2, 2, 2, 62, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 66, 2, 33, 2, 2, 2, 2, 38, 2, 2, 25, 2, 51, 36, 2, 48, 25, 2, 33, 2, 22, 2, 2, 28, 77, 52, 2, 2, 2, 2, 82, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 36, 71, 43, 2, 2, 26, 2, 2, 46, 2, 2, 2, 2, 2, 2, 88, 2, 2, 2, 2, 98, 32, 2, 56, 26, 2, 2, 2, 2, 2, 2, 2, 22, 21, 2, 2, 26, 2, 2, 2, 30, 2, 2, 51, 36, 28, 2, 92, 25, 2, 2, 2, 65, 2, 38, 2, 88, 2, 2, 2, 2, 2, 2, 2, 2, 32, 2, 2, 2, 2, 2, 32]


In [11]:
# 2. 훈련셋에서 2 제외하기
for i in range(len(x_train_all)):
    x_train_all[i] = [w for w in x_train_all[i] if w > 2]

print(x_train_all[0])

[22, 43, 65, 66, 36, 25, 43, 50, 35, 39, 38, 50, 22, 22, 71, 87, 43, 38, 76, 22, 62, 66, 33, 38, 25, 51, 36, 48, 25, 33, 22, 28, 77, 52, 82, 36, 71, 43, 26, 46, 88, 98, 32, 56, 26, 22, 21, 26, 30, 51, 36, 28, 92, 25, 65, 38, 88, 32, 32]
